In [9]:
import requests
from IPython.display import display, Markdown

base_url = 'https://dev.azure.com'
org = 'PhilipsAgile'
project = '1.0%20DC%20Digital'
team = '1.2.2%20Bodacious%20Crew'

token = 'c2liZW5kdS5kYXNAcGhpbGlwcy5jb206RWVVMWpiTlBBTXQ1dVl4aU9iOGFkeGIycERzOWRHdmkyd1JiWEJBVzVsS2ZTUmJWaU96bEpRUUo5OUJDQUNBQUFBQUdaVU5VQUFBU0FaRE8zVVNw'
headers = {'Content-Type': 'application/json', 'Authorization': 'Basic '+token}
NOT_FOUND = 'NOT_FOUND'
NEW_LINE = '<br/>'
SEPARATOR = '----------------------'

sprint_no = '2451'
iteration = '1.0 DC Digital\\'+sprint_no

In [2]:
def get_sprint_id(sprint_no):
    url = f'{base_url}/{org}/{project}/{team}/_apis/work/teamsettings/iterations?api-version=7.2-preview.1'
    
    r = requests.get(url=url, headers=headers)
    data = r.json()
    #data['count']
    sprints = data['value']
    for s in sprints:
        if s['name'] == sprint_no:
            #print('Found:' + s['id'] + ' - ' + s['name'])
            return s['id']

    return NOT_FOUND   

def get_sprint_workitems(sprint_id):
    url = f'{base_url}/{org}/{project}/{team}/_apis/work/teamsettings/iterations/{sprint_id}/workitems?api-version=7.2-preview.1'

    r = requests.get(url=url, headers=headers)
    data = r.json()
    #data['count']
    workItems = data['workItemRelations']
    return workItems

def get_workitem_details(wi):
    url = wi['target']['url']
    r = requests.get(url=url, headers=headers)
    return r.json()

def get_workitem_details_by_url(wiUrl):
    r = requests.get(url=wiUrl, headers=headers)
    return r.json()

In [3]:
# api-endpoint
url = f'{base_url}/{org}/{project}/_apis/wit/wiql?api-version=7.0'
#print(url)

wiql_query = f'SELECT [System.Id], [System.Title], [System.State], [System.WorkItemType] FROM WorkItems WHERE [System.IterationPath] = \'{iteration}\' AND [System.WorkItemType] IN (\'User Story\', \'Bug\')'
#print(wiql_query)

payload = {
  "query": "SELECT [System.Id], [System.Title], [System.State], [System.WorkItemType] FROM WorkItems WHERE [System.AreaPath] = '1.0 DC Digital\\DMEC\\D2B Clan\\D2B Platform Experience and Conversion Tribe\\D2B PEC Loyalty and Partnership Squad\\Bodacious Crew' AND [System.IterationPath] = '1.0 DC Digital\\2451' AND [System.WorkItemType] IN ('User Story', 'Bug')"
}
#print(payload)

r = requests.post(url=url, headers=headers, data=str(payload))

In [21]:
def extract_workitem_data(wiData):
    Title = wiData['fields']['System.Title']
    Type = wiData['fields']['System.WorkItemType']
    State = wiData['fields']['System.State']
    Reason = wiData['fields']['System.Reason']
    AssignedTo = wiData['fields']['System.AssignedTo']
    CreatedBy = wiData['fields']['System.CreatedBy']
    CreatedDate = wiData['fields']['System.CreatedDate']
    ChangedBy = wiData['fields']['System.ChangedBy']
    ChangedDate = wiData['fields']['System.ChangedDate']
    CommentCount = wiData['fields']['System.CommentCount']
    Title = wiData['fields']['System.Title']
    BoardColumn = wiData['fields']['System.BoardColumn']
    BoardColumnDone = wiData['fields']['System.BoardColumnDone']

    if 'Microsoft.VSTS.Scheduling.StoryPoints' in wiData['fields']:
        StoryPoints = wiData['fields']['Microsoft.VSTS.Scheduling.StoryPoints']
    else:
        StoryPoints = ''

    StateChangeDate = wiData['fields']['Microsoft.VSTS.Common.StateChangeDate']
    
    if 'Microsoft.VSTS.Common.ActivatedDate' in wiData['fields']:
        ActivatedDate = wiData['fields']['Microsoft.VSTS.Common.ActivatedDate']
    else:
        ActivatedDate = ''

    if 'Microsoft.VSTS.Common.ActivatedBy' in wiData['fields']:
        ActivatedBy = wiData['fields']['Microsoft.VSTS.Common.ActivatedBy']
    else:
        ActivatedBy = {'uniqueName':''}
        
    ResolvedDate = wiData['fields']['Microsoft.VSTS.Common.ResolvedDate']
    ResolvedBy = wiData['fields']['Microsoft.VSTS.Common.ResolvedBy']
    ClosedDate = wiData['fields']['Microsoft.VSTS.Common.ClosedDate']
    ClosedBy = wiData['fields']['Microsoft.VSTS.Common.ClosedBy']

    if 'System.Tags' in wiData['fields']:
        Tags = wiData['fields']['System.Tags']
    else:
        Tags = ''
    Description = wiData['fields']['System.Description']

    if 'Microsoft.VSTS.Common.AcceptanceCriteria' in wiData['fields']:
        AcceptanceCriteria = wiData['fields']['Microsoft.VSTS.Common.AcceptanceCriteria']
    else:
        AcceptanceCriteria = ''
        
    content = ''
    content = content + 'Type: '+ Type + NEW_LINE
    content = content + 'Title: '+ Title + NEW_LINE
    content = content + 'State: '+State + NEW_LINE
    content =  content + 'CreatedBy: '+ CreatedBy['uniqueName'] + ';  CreatedDate: '+ CreatedDate + NEW_LINE
    content =  content + 'AssignedTo: '+ AssignedTo['uniqueName'] + NEW_LINE
    content =  content + 'ChangedBy: '+ ChangedBy['uniqueName'] + ';  ChangedDate: '+ ChangedDate + NEW_LINE
    content =  content + 'ActivatedBy: '+ ActivatedBy['uniqueName'] + ';  ActivatedDate: '+ ActivatedDate + NEW_LINE
    content =  content + 'ResolvedBy: '+ ResolvedBy['uniqueName'] + ';  ResolvedDate: '+ ResolvedDate + NEW_LINE
    content =  content + 'ClosedBy: '+ ClosedBy['uniqueName'] + ';  ClosedDate: '+ ClosedDate + NEW_LINE
    content =  content + 'StoryPoints: '+ str(StoryPoints) + NEW_LINE
    content =  content + 'CommentCount: '+ str(CommentCount) + NEW_LINE
    content =  content + 'Tags: '+ Tags + NEW_LINE
    content =  content + 'Description: '+ Description + NEW_LINE 
    content =  content + 'AcceptanceCriteria: '+ AcceptanceCriteria + NEW_LINE 
    return content

In [22]:
workItems = r.json()['workItems']
content = ''
for wi in workItems:
    wiData = get_workitem_details_by_url(wi['url'])
    content =  content +  extract_workitem_data(wiData) + SEPARATOR 
    #print(wiData)    

In [24]:
filename = sprint_no+'.txt'
f = open(filename, 'a')
f.write(content)
f.close()
print('Written to file: '+filename)

Written to file: 2451.txt


In [23]:
#content = extract_workitem_data(wiData)
display(Markdown(content))

Type: User Story<br/>Title: New Order Overview - Super User reusable Component to be used in both Order Overview Compoenents(Spill)<br/>State: 5 Accepted<br/>CreatedBy: partner.Shreelakshmi.Venugopal@philips.com;  CreatedDate: 2024-10-22T13:21:18.753Z<br/>AssignedTo: partner.Sreekanth.M@philips.com<br/>ChangedBy: partner.Vipashchit.Kashyap@philips.com;  ChangedDate: 2025-02-10T07:12:58.32Z<br/>ActivatedBy: partner.Sreekanth.M@philips.com;  ActivatedDate: 2024-10-25T05:59:09.35Z<br/>ResolvedBy: partner.Chandrashekhar.Badgujar@philips.com;  ResolvedDate: 2025-01-03T08:57:36.923Z<br/>ClosedBy: partner.Shreelakshmi.Venugopal@philips.com;  ClosedDate: 2025-01-09T05:18:34.403Z<br/>StoryPoints: 2.0<br/>CommentCount: 3<br/>Tags: CP2025Q1; SIT Deployed<br/>Description: Super User reusable Component to be used in both Order Overview Components<br><br><br/>AcceptanceCriteria: <div>Super User reusable Component to be used in both Order Overview Compoenents<br> </div><br/>----------------------Type: Bug<br/>Title: ST || Vulnerable version of the library 'DOMPurify' found<br/>State: 5 Accepted<br/>CreatedBy: partner.Jayesh.Thakare@philips.com;  CreatedDate: 2024-10-24T10:42:30.783Z<br/>AssignedTo: partner.Jayesh.Thakare@philips.com<br/>ChangedBy: partner.Jayesh.Thakare@philips.com;  ChangedDate: 2024-12-24T05:13:23.907Z<br/>ActivatedBy: ;  ActivatedDate: <br/>ResolvedBy: partner.raju.sharma@philips.com;  ResolvedDate: 2024-12-19T13:28:13.97Z<br/>ClosedBy: partner.Jayesh.Thakare@philips.com;  ClosedDate: 2024-12-24T05:13:23.907Z<br/>StoryPoints: <br/>CommentCount: 1<br/>Tags: Mindtree_ST; Security; SFFT<br/>Description: <div>Vulnerable version of the library 'DOMPurify' found<br> </div><br/>AcceptanceCriteria: <br/>----------------------Type: User Story<br/>Title: SF Optimizer - Critical Permissions on Local Admin Profile - Analysis and documentation<br/>State: 5 Accepted<br/>CreatedBy: Aarti.Jagtap@philips.com;  CreatedDate: 2024-11-08T14:02:10.07Z<br/>AssignedTo: partner.Soundarya.G@philips.com<br/>ChangedBy: partner.Vipashchit.Kashyap@philips.com;  ChangedDate: 2025-02-10T07:11:54.117Z<br/>ActivatedBy: partner.Soundarya.G@philips.com;  ActivatedDate: 2024-12-18T05:38:23.237Z<br/>ResolvedBy: partner.Soundarya.G@philips.com;  ResolvedDate: 2024-12-19T12:30:17.653Z<br/>ClosedBy: partner.Shreelakshmi.Venugopal@philips.com;  ClosedDate: 2025-01-09T05:18:13.62Z<br/>StoryPoints: 1.0<br/>CommentCount: 1<br/>Tags: Analysis story; CP2025Q1<br/>Description: <div>What capabilities we offer to local admin </div><div>which access and level we need to achieve those capabilities </div><div>from above point compare what extra access we have added to local admin profile / PS </div><div>then we can decide what to reduce <br><br><br> </div><br/>AcceptanceCriteria: <div>TBD <br><br><div style="box-sizing:border-box;">What capabilities we offer to local admin </div><div style="box-sizing:border-box;">which access and level we need to achieve those capabilities </div><div style="box-sizing:border-box;">from above point compare what extra access we have added to local admin profile / PS </div><div style="box-sizing:border-box;">then we can decide what to reduce<br style="box-sizing:border-box;"><br style="box-sizing:border-box;"> </div><br> </div><br/>----------------------Type: User Story<br/>Title: Working E-E Test Data identification and finalization SIT and UAT<br/>State: 5 Accepted<br/>CreatedBy: Aarti.Jagtap@philips.com;  CreatedDate: 2024-12-02T08:06:47.99Z<br/>AssignedTo: partner.urmaliya.somdev@philips.com<br/>ChangedBy: partner.Vipashchit.Kashyap@philips.com;  ChangedDate: 2025-02-10T07:14:04.31Z<br/>ActivatedBy: ;  ActivatedDate: <br/>ResolvedBy: partner.urmaliya.somdev@philips.com;  ResolvedDate: 2025-01-02T05:45:53.607Z<br/>ClosedBy: partner.Shreelakshmi.Venugopal@philips.com;  ClosedDate: 2025-01-09T05:18:59.027Z<br/>StoryPoints: 0.0<br/>CommentCount: 2<br/>Tags: CP2025Q1; Test Only<br/>Description: <div><span>In SIT and UAT environments Identify a set of Partner Master Accounts and Delegate, Employee and ICM users respective where end to end functionality and learning life cycle works till November release.<br></span> </div><div><span><a href="#" data-vss-mention="version:2.0,7b905768-6894-626f-8564-ed690045286c">@Somdev, Urmaliya (Partner)</a>&nbsp;<a href="#" data-vss-mention="version:2.0,d73e0f0e-4786-6282-af20-c9c0a2bd4a01">@P, Mithra (Partner)</a> </span> </div><div><div><br> </div><span>Use below wiki link to document this <a href="https://dev.azure.com/PhilipsAgile/1.0%20DC%20Digital/_wiki/wikis/1.0-DC-Digital.wiki/21757/Identified-Test-Data">https://dev.azure.com/PhilipsAgile/1.0%20DC%20Digital/_wiki/wikis/1.0-DC-Digital.wiki/21757/Identified-Test-Data</a></span><br> </div><br/>AcceptanceCriteria: <div><span>SIT test data [5 Partner Master Accounts are identified as per our testing requirements which will cover end-end journey]<br></span> </div><div><div>documented in wiki<br> </div><div>UAT test data [10 Partner Master Accounts are identified as per our testing requirements which will cover end-end journey]<br> </div><span>documented in wiki</span><br> </div><br/>----------------------Type: User Story<br/>Title: Note Hybris and User Registration - Partner : Code Details<br/>State: 5 Accepted<br/>CreatedBy: Aarti.Jagtap@philips.com;  CreatedDate: 2024-12-13T10:52:36.273Z<br/>AssignedTo: partner.Chandrashekhar.Badgujar@philips.com<br/>ChangedBy: partner.Vipashchit.Kashyap@philips.com;  ChangedDate: 2025-02-10T07:14:30.4Z<br/>ActivatedBy: partner.Chandrashekhar.Badgujar@philips.com;  ActivatedDate: 2024-12-17T15:37:24.807Z<br/>ResolvedBy: partner.Chandrashekhar.Badgujar@philips.com;  ResolvedDate: 2025-01-03T05:48:19.803Z<br/>ClosedBy: partner.Shreelakshmi.Venugopal@philips.com;  ClosedDate: 2025-01-09T05:19:28.01Z<br/>StoryPoints: 1.0<br/>CommentCount: 2<br/>Tags: CP2025Q1; Documentation<br/>Description: <div><a href="#" data-vss-mention="version:2.0,b3d0b2a4-f02a-63bd-a86d-3bb119c92e60">@Chandrashekhar Badgujar</a>&nbsp;please note all scenarios and code flow for Hybris and User registration API Partner - MyP4P custom code </div><div>Flow diagram will help better and an excel document </div><br/>AcceptanceCriteria: <div><span style="display:inline !important;">noted all scenarios and code flow for Hybris and User registration API - MyP4P custom code</span><br> </div><div><span style="display:inline !important;">flow diagram and excel document</span> </div><br/>----------------------Type: User Story<br/>Title: Improvements - My Philips Dashboard Content Section - part2<br/>State: 5 Accepted<br/>CreatedBy: partner.raju.sharma@philips.com;  CreatedDate: 2024-12-17T05:58:24.093Z<br/>AssignedTo: partner.raju.sharma@philips.com<br/>ChangedBy: partner.Vipashchit.Kashyap@philips.com;  ChangedDate: 2025-02-10T07:11:30.147Z<br/>ActivatedBy: partner.raju.sharma@philips.com;  ActivatedDate: 2024-12-17T06:07:03.587Z<br/>ResolvedBy: partner.raju.sharma@philips.com;  ResolvedDate: 2025-01-06T12:02:46.003Z<br/>ClosedBy: partner.Shreelakshmi.Venugopal@philips.com;  ClosedDate: 2025-01-09T04:52:24.517Z<br/>StoryPoints: 5.0<br/>CommentCount: 2<br/>Tags: CP2025Q1; SIT Deployed<br/>Description: <div><span data-rendered-mention=people><span class=mention-preview-person>@Chandrashekhar Badgujar</span></span> </div><div style="box-sizing:border-box;"> </div><div style="box-sizing:border-box;">1- Add a search box for country&nbsp; </div><div style="box-sizing:border-box;">2- Select all the countries automatically if I select a particular market (if I manage more than a market) </div><div style="box-sizing:border-box;"><br> </div><div style="box-sizing:border-box;"><br style="box-sizing:border-box;"> </div><div style="box-sizing:border-box;"><img src="https://dev.azure.com/PhilipsAgile/bd5965c1-53bb-4698-915e-492450ee1ff0/_apis/wit/attachments/39cfda56-36db-4df2-af25-b87fa1c93b57?fileName=image.png" alt=Image width=637 height=487 style="box-sizing:border-box;max-width:100%;align-self:center;width:637px;"><br style="box-sizing:border-box;"> </div><div style="box-sizing:border-box;"><br style="box-sizing:border-box;"> </div><div style="box-sizing:border-box;"><br style="box-sizing:border-box;"> </div><div style="box-sizing:border-box;"><br style="box-sizing:border-box;"> </div><div><br> </div><br/>AcceptanceCriteria: <div><div style="box-sizing:border-box;">1- Add a search box for country&nbsp; </div><div style="box-sizing:border-box;">2- Select all the countries automatically if I select a particular market (if I manage more than a market) </div><div style="box-sizing:border-box;"><br style="box-sizing:border-box;"> </div><div style="box-sizing:border-box;"><br> </div><div style="box-sizing:border-box;"><img src="https://dev.azure.com/PhilipsAgile/bd5965c1-53bb-4698-915e-492450ee1ff0/_apis/wit/attachments/39cfda56-36db-4df2-af25-b87fa1c93b57?fileName=image.png" alt=Image width=637 height=487 style="box-sizing:border-box;max-width:100%;align-self:center;width:637px;"><br style="box-sizing:border-box;"> </div><div style="box-sizing:border-box;"><br style="box-sizing:border-box;"> </div><div style="box-sizing:border-box;"><br style="box-sizing:border-box;"> </div><div style="box-sizing:border-box;"><br style="box-sizing:border-box;"> </div><br><br> </div><div> </div><div><br> </div><div><br> </div><br/>----------------------Type: User Story<br/>Title: Sailpoint Application version Upgrade<br/>State: 5 Accepted<br/>CreatedBy: partner.raju.sharma@philips.com;  CreatedDate: 2024-12-17T10:26:48.24Z<br/>AssignedTo: partner.Sweta.Prajapati@philips.com<br/>ChangedBy: partner.Vipashchit.Kashyap@philips.com;  ChangedDate: 2025-02-10T07:15:08.58Z<br/>ActivatedBy: partner.Sweta.Prajapati@philips.com;  ActivatedDate: 2024-12-19T05:43:03.54Z<br/>ResolvedBy: partner.Sweta.Prajapati@philips.com;  ResolvedDate: 2024-12-20T02:36:00.16Z<br/>ClosedBy: partner.Shreelakshmi.Venugopal@philips.com;  ClosedDate: 2025-01-09T05:14:39.983Z<br/>StoryPoints: 1.0<br/>CommentCount: 1<br/>Tags: CP2025Q1; Test Only<br/>Description: <div>Sailpoint Application version Upgrade v8.4 P1 - QA environment - Require Support for UAT Testing &nbsp;<br> </div><br/>AcceptanceCriteria: <div>Sailpoint Application version Upgrade v8.4 P1 - QA environment - Require Support for UAT Testing &nbsp;<br> </div><br/>----------------------Type: User Story<br/>Title: KT from Sweta<br/>State: 5 Accepted<br/>CreatedBy: partner.raju.sharma@philips.com;  CreatedDate: 2024-12-17T10:49:38.167Z<br/>AssignedTo: partner.Sweta.Prajapati@philips.com<br/>ChangedBy: partner.Shreelakshmi.Venugopal@philips.com;  ChangedDate: 2025-01-09T05:24:45.27Z<br/>ActivatedBy: partner.Sweta.Prajapati@philips.com;  ActivatedDate: 2024-12-18T08:31:15.493Z<br/>ResolvedBy: partner.Sweta.Prajapati@philips.com;  ResolvedDate: 2024-12-19T08:16:27.323Z<br/>ClosedBy: partner.Shreelakshmi.Venugopal@philips.com;  ClosedDate: 2025-01-09T05:24:45.27Z<br/>StoryPoints: 1.0<br/>CommentCount: 1<br/>Tags: <br/>Description: <div>KT from Sweta<br> </div><br/>AcceptanceCriteria: <div>KT from Sweta<br><a href="#" data-vss-mention="version:2.0,380d10d5-3d4b-61ca-81ac-ae522b56cfa2">@Prajapati, Sweta (Partner)</a>&nbsp;please add the list of US/module which you are going to give KT. </div><br/>----------------------Type: User Story<br/>Title: Test Execution for sprints 2449&2451User stories<br/>State: 5 Accepted<br/>CreatedBy: partner.Mithra.P@philips.com;  CreatedDate: 2024-12-18T08:50:06.99Z<br/>AssignedTo: partner.Mithra.P@philips.com<br/>ChangedBy: partner.Vipashchit.Kashyap@philips.com;  ChangedDate: 2025-02-10T07:10:59.5Z<br/>ActivatedBy: partner.Mithra.P@philips.com;  ActivatedDate: 2024-12-20T06:57:05.36Z<br/>ResolvedBy: partner.Mithra.P@philips.com;  ResolvedDate: 2025-01-03T04:42:39.67Z<br/>ClosedBy: partner.Mithra.P@philips.com;  ClosedDate: 2025-01-03T04:58:51.93Z<br/>StoryPoints: 0.0<br/>CommentCount: 0<br/>Tags: CP2025Q1; Test Only<br/>Description: <div><div style="box-sizing:border-box;">Below user story from 2249 and 2451 for Execution </div><div><br> </div><span style="font-family:&quot;Aptos Narrow&quot;, sans-serif;font-size:14.6667px;display:inline !important;">1.My Group selection for first time login Users (Non-Partners)</span><br><span style="font-family:&quot;Aptos Narrow&quot;, sans-serif;font-size:14.6667px;display:inline !important;">&nbsp;2.Improvements - My Philips Dashboard Content Section</span><br>3.New Order Overview - Super User reusable Component to be used in both Order Overview Compoenents(Spill) </div><div>4.Permissions segregation for HCP-ICP and Hybris users - Part 2<br> </div><div><br><br> </div><br/>AcceptanceCriteria: <table width=467 style="border-collapse:collapse;width:351pt;">

 <colgroup><col width=357 style="width:268pt;">
 <col width=110 style="width:83pt;">
 </colgroup><tbody><tr height=41 style="box-sizing:border-box;">
  <td height=41 width=357 style="padding-top:1px;padding-right:1px;padding-left:1px;color:black;font-size:11pt;font-family:&quot;Aptos Narrow&quot;, sans-serif;vertical-align:bottom;border:none;text-align:center;border:0.5pt solid windowtext;background:white;border:1px solid rgb(212, 212, 212);width:268pt;box-sizing:border-box;">User stories </td>
  <td width=110 style="padding-top:1px;padding-right:1px;padding-left:1px;color:black;font-size:11pt;font-family:&quot;Aptos Narrow&quot;, sans-serif;vertical-align:bottom;border:none;text-align:center;border:0.5pt solid windowtext;background:white;border:1px solid rgb(212, 212, 212);border-left:none;width:83pt;box-sizing:border-box;">Execution
  Status&nbsp; </td>
 </tr>
 <tr height=21 style="box-sizing:border-box;">
  <td height=21 width=357 style="padding-top:1px;padding-right:1px;padding-left:1px;color:black;font-size:11pt;font-family:&quot;Aptos Narrow&quot;, sans-serif;vertical-align:bottom;border:none;border:0.5pt solid windowtext;background:white;border:1px solid rgb(212, 212, 212);border-top:none;width:268pt;box-sizing:border-box;"><span style="box-sizing:border-box;">Improvements
  - My Philips Dashboard Content Section</span> </td>
  <td width=110 style="padding-top:1px;padding-right:1px;padding-left:1px;color:black;font-size:11pt;font-family:&quot;Aptos Narrow&quot;, sans-serif;vertical-align:bottom;border:none;text-align:center;border:0.5pt solid windowtext;background:white;border:1px solid rgb(212, 212, 212);border-top:none;border-left:none;width:83pt;box-sizing:border-box;">Dev </td>
 </tr>
 <tr height=20 style="box-sizing:border-box;">
  <td rowspan=2 height=40 width=357 style="padding-top:1px;padding-right:1px;padding-left:1px;color:black;font-size:11pt;font-family:&quot;Aptos Narrow&quot;, sans-serif;vertical-align:bottom;border:none;border:0.5pt solid windowtext;background:white;border:1px solid rgb(212, 212, 212);border-top:none;width:268pt;box-sizing:border-box;"><span style="box-sizing:border-box;display:inline;">&nbsp;My Group selection for first time login Users
  (Non-Partners)</span> </td>
  <td rowspan=2 width=110 style="padding-top:1px;padding-right:1px;padding-left:1px;color:black;font-size:11pt;font-family:&quot;Aptos Narrow&quot;, sans-serif;vertical-align:bottom;border:none;text-align:center;border:0.5pt solid windowtext;background:white;border:1px solid rgb(212, 212, 212);border-top:none;width:83pt;">Completed </td>
 </tr>
 <tr height=20>
 </tr>
 <tr height=40>
  <td height=40 width=357 style="padding-top:1px;padding-right:1px;padding-left:1px;color:black;font-size:11pt;font-family:&quot;Aptos Narrow&quot;, sans-serif;vertical-align:bottom;border:none;border:0.5pt solid windowtext;border:1px solid rgb(212, 212, 212);border-top:none;width:268pt;">New Order Overview - Super User reusable Component to be used in
  both Order Overview Compoenents </td>
  <td width=110 style="padding-top:1px;padding-right:1px;padding-left:1px;color:black;font-size:11pt;font-family:&quot;Aptos Narrow&quot;, sans-serif;vertical-align:bottom;border:none;text-align:center;border:0.5pt solid windowtext;background:white;border:1px solid rgb(212, 212, 212);border-top:none;border-left:none;width:83pt;box-sizing:border-box;">Dev </td>
 </tr>
 <tr height=20 style="">
  <td height=20 style="padding-top:1px;padding-right:1px;padding-left:1px;color:black;font-size:11pt;font-family:&quot;Aptos Narrow&quot;, sans-serif;vertical-align:bottom;border:none;border:0.5pt solid windowtext;border:1px solid rgb(212, 212, 212);border-top:none;">Permissions
  segregation for HCP-ICP and Hybris users -<span style="">&nbsp;</span> </td>
  <td style="padding-top:1px;padding-right:1px;padding-left:1px;color:black;font-size:11pt;font-family:&quot;Aptos Narrow&quot;, sans-serif;vertical-align:bottom;border:none;text-align:center;border:0.5pt solid windowtext;border:1px solid rgb(212, 212, 212);border-top:none;border-left:none;">Completed </td>
 </tr></tbody></table><br><br/>----------------------

In [47]:
sprint_id = get_sprint_id(sprint_no)
print(sprint_id)
workItems = get_sprint_workitems(sprint_id)
print(len(workItems))
items = []
for wi in workItems:
    #print(wi['target']['url'])
    w = get_workitem_details(wi)
    #print(w)
    if w['fields']['System.IterationPath'] == iteration:
        items.append(w)

print('Total iems = '+str(len(items)))
for w in items:
    print(str(w['id'])+ ' - ' + str(w['rev'])+ ' - ' + str(w['fields']['System.IterationPath'])+ ' - ' + str(w['fields']['System.State']))
    print(w)

bed81a8b-dfba-4ceb-837f-04fc12a02aff
30
